In [ ]:
#instrukce
#skript vyžaduje mít xml dokumenty na jednom místě
#na stejném místě mít i queries_cs
#cestu k těmto dokumentům je potřeba na jednom místě zadat
#tato proměnná se jmenuje "path"

In [ ]:
#1. nejdřív sebrat všechny dokumenty
#2. tokenizovat
#3. normalizovat
#4. udělat inverted index

In [ ]:
#funkce pro konverzi stringů do tokenů v listech
def Convert(string):
    li = list(string.split(" "))
    return li

In [ ]:
#funkce pro vytvoření inverted indexu
from collections import defaultdict
def create_index (data):
    index = defaultdict(list)
    for i, tokens in enumerate(data):
        for token in tokens:
             index[token].append(i)
    return index

In [ ]:
#zpracování dat
import html
import os, sys
from lxml import etree
import xml.etree.ElementTree as ET

kolekce = []
hashed = []
text_ready = []

#v tomto dictu je pořadí "DOC" jako key a "DOCID" jako value
doc_ID_dict = {}

# counters
counter_child = 0
counter_node = 0
counter_doc_ID = 0
counter_file = 0

path = "/home/mazurri/InfoRetrieval"
directory = os.listdir(path)

for file in directory:
    counter_child = 0
    counter_file = counter_file + 1
    test = os.path.splitext(file)
    #test, předpokládá se, že queries_cs bude ve stejné složce
    if test[0] == "queries_cs":
        continue      
    elif test[1] == ".xml":
        print(file)
        #kvůli erroru s unescaped characters "&"
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        for child in root:
            
            ### populování doc_ID_dictu pro identifikaci dokumentu
            doc_ID = child.find("DOCID").text
            doc_ID_dict[counter_doc_ID] = doc_ID
            #nejdřív append, potom inkrement
            counter_doc_ID = counter_doc_ID + 1
            
            for node in child:
                #prvni 3 nodes jsou vzdy stejne a neobsahuji text, který by bylo vhodné využít
                if counter_node < 3:
                    counter_node = counter_node + 1
                    continue
                else:
                    text = root[counter_child][counter_node].text
                    
                    counter_node = counter_node + 1

                    ### normalizace ###
                    #vše do lower case
                    text = text.lower()

                    #interpunkční znaménka
                    text = text.replace("\n", " ")
                    text = text.replace("?", " ")
                    text = text.replace("!", " ")
                    #tady u tečky možná by bylo dobré použít regulární výrazy aby to nesmazalo tečku co je u čísla
                    text = text.replace(".", " ")
                    #tady u tečky možná by bylo dobré použít regulární výrazy aby to nesmazalo čárku co je u čísla
                    text = text.replace(",", " ")
                    text = text.replace(" - ", " ")
                    
                    # tokenizace do listu podle předdefinované funkce
                    text = Convert(text)
                    
                    ### hashovaní stringů ###
                    for i in text:
                        hashed.append(hash(i))
                    
            ### vytvoření setů pro eliminaci duplikátů ###
            hashed_set = set(hashed)
            hashed = []

            ### přidání setu do kolekce ###
            kolekce.append(hashed_set)
            hashed_set = None
  
            counter_node = 0
            counter_child = counter_child + 1
    else:
        continue

In [ ]:
#konzultace
#-style guide python: různí lidé následují různé guides (zejména jména proměnných, komentování), 
#jde i o to jestli to dělám pro sebe, nebo pro jiné
#-dtd -> xml parser (etree), může se stát, že některé položky neodpovídají dtd (s tím parser má problém),
#jsou to data z praxe, takhle to vypadlo, takže potřeba to nějak ošetřit
#-část mého kódu kde dělám-> recover=TRUE, tak tam se možná řeší víc chyb než jen &
#-dělat counter manuálně je low-level
#-regulární výrazy v normalizaci ->dá se napsat do jednoho výrazu
#-parsování dotazů (to je to co jsem dělal s queries), náš úkol
#-NOT je unární operátor
#-nahradit "AND NOT" například AND_NOT a pracovat s tím jako s jedním operátorem
#-typická úloha pro princip "rozděl a panuj", měl bych list jednotlivých slov v dotaze, neřešil bych jestli je něco
#operátor, nebo ne
#-PAVEL AND PETR (3 tokeny), funkce "vyřeš dotaz", dva parametry, který ukazují (pointers) 1-3, vyřeš dotaz na
#pozici 1 až 3, čímž vyřeší celý dotaz, jde zleva doprava, až narazí na operátor, tak rozdělí na sub problémy
#vyřeší 1 a pak 3

In [ ]:
inverted_index = create_index(kolekce)

In [ ]:
### INPUT ###

In [ ]:
operatory = ["AND", "OR", "AND_NOT"]

In [ ]:
def findWithinIndex (data):
    data_new = data
    token_index = 0
    for i in data_new:
        text = None
        if i in operatory:
            print("je to operator")
            print(token_index)
        else:
            data_new[token_index] = inverted_index[hash(i.lower())]
            print("není to operátor")
            print(token_index)
        token_index = token_index + 1
        #print(token_index)
    return data_new

In [ ]:
def better_findDocuments (data):
    #['ohrožený', 'OR', 'chráněný', 'AND', 'druh']
    query_docs = []
    query_docs_temp = []
    for token_index, token in enumerate(data):
        
        if token_index == 0:#funguje
            for item in token: #je to seznam, takže pro každý element v listu
                query_docs.append(item) #přidat do query_docs
        elif token_index % 2 !=0:
            
            next_token = data[token_index+1]
            #je to operator
            #je delší token, nebo query_docs?
            #print(query_docs)
            #print(token)
            
            short_list = min(query_docs, next_token, key=len)
            long_list = max(query_docs, next_token, key=len)
            #print("short_list je kratší. Má délku: ", len(short_list))
            #print("longer_list je delší. Má délku: ", len(longer_list))
            #print(short_list)
            
            if token == "OR":
                query_docs = sorted(list(set(query_docs+next_token))) #připoj co následuje za OR do query_docs
            
            if token == "AND":
                query_docs = sorted(list(set(short_list).intersection(set(long_list))))
            
            if token == "AND_NOT":
                query_docs = sorted(list(set(short_list)-set(long_list)))               
                
        else:
            print("není to operátor")
            #print(token)
            continue
    return query_docs

In [ ]:
# counters
import re


counter_query_ID = 0


operatory = ["AND", "OR", "AND_NOT"]

for file in directory:
    test = os.path.splitext(file)
    #print(test[0])
    if test[0] == "queries_cs":
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser = parser)
        root = tree.getroot()
        
        for counter_query_ID, child in enumerate(root):
            print("tohle je query číslo", counter_query_ID)
            ### pro pojmenování výsledného filu
            doc_NUM = child.find("num").text
            query = child.find("query").text            
            query = re.sub("AND NOT", "AND_NOT", query)
            print(doc_NUM)



            converted_query = Convert(query)
            print("query je", converted_query)
            #jedna query měla prázdný znak, házelo to error
            if "" in converted_query:
                converted_query.remove("")
            #tady udělat funci, která to všecko zmákne
            #mám vždy na druhé pozici operátor, chci získat seznam z inverted indexu pro vše co není operátor
            hashed_query = findWithinIndex(converted_query)
            vysledek =  better_findDocuments(hashed_query)
            print("výsledek je", vysledek)
            
            new_list = []
            for item in vysledek:
                if item > len(doc_ID_dict):
                    new_list.append(item)
                else:
                    new_list.append(doc_ID_dict[item])
            
            full_str = ' '.join([str(elem) for elem in new_list])

            
            with open("vysledky.txt", "a+") as final_file:
            # Move read cursor to the start of file.
                final_file.seek(0)
            # If file is not empty then append '\n'
                data = final_file.read(100)
                if len(data) > 0 :
                    final_file.write("\n")
                # Append text at the end of file
                final_file.write("\n")
                final_file.write("\n")
                final_file.write(doc_NUM)
                final_file.write("\n")
                final_file.write(full_str)

            new_list = []

    else:
        continue